<a href="https://colab.research.google.com/github/YAMINI-AKULA/LINGUA_research/blob/main/Copy_of_llmlingua.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLMLingua Document Compressor

>[LLMLingua](https://github.com/microsoft/LLMLingua) utilizes a compact, well-trained language model (e.g., GPT2-small, LLaMA-7B) to identify and remove non-essential tokens in prompts. This approach enables efficient inference with large language models (LLMs), achieving up to 20x compression with minimal performance loss.

This notebook shows how to use LLMLingua as a compressor.

In [ ]:
%pip install --upgrade --quiet  llmlingua accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 69.9 MB/s eta 0:00:00


In [24]:
# Helper function for printing docs


def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [35]:
def pretty_print_docs(docs):
    for i, d in enumerate(docs):
        content = d.page_content.strip().split('\n')
        print(f"Document {i+1}:")
        for line in content:
            while len(line) > 0:
                print(line[:100])
                line = line[100:]



# **Importing the dataset**

In [2]:
#importing the dataset , CCN mail from hugging face
!pip install datasets
from datasets import load_dataset
ds = load_dataset("abisee/cnn_dailymail", "1.0.0")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 2

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

# **Creating a random sample of 300 in the train data**

In [3]:
import random
from datasets import load_dataset, Dataset

random.seed(123)

# Access the 'train' split
train_ds = ds['train']


# Randomly sample 300 examples from the train dataset
sampled_indices = random.sample(range(len(train_ds)), 300)
sampled_articles = [train_ds[idx]['article'] for idx in sampled_indices]

# Create a new Dataset with only the 'article' column
sample_train_300 = Dataset.from_dict({"article": sampled_articles})

#sample_train_300.head()
# Print the first few examples to verify
for idx in range(5):  # Print the first 5 examples
    print("Example", idx + 1)
    print("Article:", sample_train_300 ['article'][idx])
    print("="*50)


Example 1
Article: Kabul, Afghanistan (CNN) -- Two Americans and three Afghan soldiers died in a firefight that might have involved insurgent forces, the International Security Assistance Force said Sunday. One of the U.S. deaths was a soldier; the other was a civilian, U.S. officials said. The coalition first said the clash Saturday was the latest in a spate of insider attacks involving Afghan forces, which have killed more than 50 NATO troops this year. Later, the coalition called that into question. It appears the clash in Sydabad district might have been provoked by insurgent fire, said Lt. Gen. Adrian Bradshaw, the ISAF deputy commander, at a news conference. Marine killed in Afghanistan predicted insider attack that lead to his death . The firefight took place at a temporary checkpoint being operated by ISAF troops in Wardak province, west of Kabul. An Afghan National Army, or ANA unit, was nearby, Bradshaw said. "According to ISAF and ANA reporting, after a short conversation to

In [ ]:
# Check the type of the loaded dataset
print(type(sample_train_300))

<class 'datasets.arrow_dataset.Dataset'>


## Set up the base vector store retriever
Let's start by initializing a simple vector store retriever and storing the data.

In [4]:
! pip install langchain_openai
!pip install langchain_community



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.5/327.5 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 59.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.4 MB/s eta 0:00:00


In [9]:
import os

from google.colab import userdata
userdata.get('OPENAI_API_KEY')

# Get the API key from userdata
openai_api_key = userdata.get('OPENAI_API_KEY')

# Set the API key as an environment variable
os.environ["OPENAI_API_KEY"] = openai_api_key


In [7]:
!pip install Chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 8.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 4

In [10]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from datasets import Dataset

# Initialize OpenAIEmbeddings model
embedding = OpenAIEmbeddings(model="text-embedding-ada-002")

# Extract texts from the dataset
texts = sample_train_300['article']

# Generate embeddings for all texts
embeddings = []
for text in texts:
    embedding_result = embedding.embed_query(text)
    embeddings.append(embedding_result)



In [11]:
# Display the first 5 embedding results
for i in range(5):
    #print(f"Text: {texts[i]}")
    print(f"Embedding: {embeddings[i]}")
    print()

Embedding: [-0.022279400378465652, -0.005733958911150694, -0.002806259086355567, -0.01166484784334898, -0.0009050596272572875, -0.0019118665950372815, -0.019220413640141487, -0.007555565796792507, -0.006219720467925072, -0.008310466073453426, 0.038414571434259415, 0.02281767688691616, -0.003958302550017834, 0.008146356791257858, -0.0035414663143455982, -0.002957239281386137, 0.026572484523057938, 0.005560003686696291, -0.0011594281531870365, -0.014979843981564045, -0.02138664945960045, 0.028095412999391556, 0.00011908139276783913, 0.027596523985266685, -0.002138336654752493, 0.0030245238449424505, -0.009222910739481449, -0.014717270620167255, 0.02649371325969696, -0.03258542716503143, -0.009321375750005245, -0.027859097346663475, -0.03371449559926987, -0.005392612889409065, -0.01719859428703785, -0.01337157841771841, 0.011901164427399635, -0.026703771203756332, 0.015885723754763603, -0.02308025024831295, 0.043797336518764496, -0.0040009706281125546, 0.014966716058552265, -0.00245506642

**testing for a query against the vector store**

In [20]:
# Initialize Chroma vector store (example, adjust based on actual implementation)

chroma = Chroma(embedding_function=embedding)  # Pass the embedding model here

# Since IDs are not specified (assuming they are optional), pass None for IDs
ids = None

# Add texts and embeddings to Chroma
chroma.add_texts(texts=texts, embeddings=embeddings, metadatas=metadata, ids=ids)

# Create a retriever (if needed)
retriever = chroma.as_retriever(search_kwargs={"k": 20})


In [36]:
query = "how many soldiers died in a firefight that might have involved insurgent forces?"
docs = retriever.invoke(query)
pretty_print_docs(docs)

Document 1:
Kabul, Afghanistan (CNN) -- Two Americans and three Afghan soldiers died in a firefight that might h
ave involved insurgent forces, the International Security Assistance Force said Sunday. One of the U
.S. deaths was a soldier; the other was a civilian, U.S. officials said. The coalition first said th
e clash Saturday was the latest in a spate of insider attacks involving Afghan forces, which have ki
lled more than 50 NATO troops this year. Later, the coalition called that into question. It appears 
the clash in Sydabad district might have been provoked by insurgent fire, said Lt. Gen. Adrian Brads
haw, the ISAF deputy commander, at a news conference. Marine killed in Afghanistan predicted insider
 attack that lead to his death . The firefight took place at a temporary checkpoint being operated b
y ISAF troops in Wardak province, west of Kabul. An Afghan National Army, or ANA unit, was nearby, B
radshaw said. "According to ISAF and ANA reporting, after a short conversation 

## Doing compression with LLMLingua
Now let’s wrap our base retriever with a `ContextualCompressionRetriever`, using `LLMLinguaCompressor` as a compressor.

In [27]:
!pip install llmlingua

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 6.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [28]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain_community.document_compressors import LLMLinguaCompressor
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0)

compressor = LLMLinguaCompressor(model_name="openai-community/gpt2", device_map="cpu")#defining the compression using the small lm
compression_retriever = ContextualCompressionRetriever( # initializing a compressed retriever, which combines the base retriver+compressor
    base_compressor=compressor, base_retriever=retriever
)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1438 > 1024). Running this sequence through the model will result in indexing errors


Document 1:

#ref0#> Kabul ( Americans and three Afghan soldiers died in a firefight that might have involved insurgent, International. One the.; the civilian. Saturday was in of involving forces have killed more NATO. Later, the called. inabad might provoked by,.. Adrianshaw, ISAF commander in Afghanistan attack that to his death. firefight a by in province, National, or AN unit was nearbyAccording ISAF and reporting, short betweenAF personnel firing which resultedAF and the of his civilian." Neither the the rounds InvestigatorsAF and the Afghan to establish the exchange provoked by," said," but "there is a of," In statement clashS. Afghanistan to, a CNN The four prayers lady " of those who sacrifice so this including the those have made the said, Afghan -- or wearing -- on ISAFents in attack in Army uniforms.-called- been concerns of NATO in ""-oding used by military, in blue to international and green allied " way commitment president allies to complete transition to by senior leade

In [37]:
compressed_docs = compression_retriever.invoke(
    "how many soldiers died in a firefight that might have involved insurgent forces?"
)
pretty_print_docs(compressed_docs)

Document 1:
#ref0#> Kabul ( Americans and three Afghan soldiers died in a firefight that might have involved ins
urgent, International. One the.; the civilian. Saturday was in of involving forces have killed more 
NATO. Later, the called. inabad might provoked by,.. Adrianshaw, ISAF commander in Afghanistan attac
k that to his death. firefight a by in province, National, or AN unit was nearbyAccording ISAF and r
eporting, short betweenAF personnel firing which resultedAF and the of his civilian." Neither the th
e rounds InvestigatorsAF and the Afghan to establish the exchange provoked by," said," but "there is
 a of," In statement clashS. Afghanistan to, a CNN The four prayers lady " of those who sacrifice so
 this including the those have made the said, Afghan -- or wearing -- on ISAFents in attack in Army 
uniforms.-called- been concerns of NATO in ""-oding used by military, in blue to international and g
reen allied " way commitment president allies to complete transition to by seni

## QA generation without the compressor

In [33]:
from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

In [34]:
chain.invoke({"query": query})

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 16385 tokens. However, your messages resulted in 17675 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

## QA generation with LLMLingua

We can see what it looks like to use this in the generation step now

In [31]:
from langchain.chains import RetrievalQA

compressed_chain = RetrievalQA.from_chain_type(llm=llm, retriever=compression_retriever)

In [32]:
compressed_chain.invoke({"query": query})

{'query': 'how many soldiers died in a firefight that might have involved insurgent forces?',
 'result': 'In the firefight mentioned in the context, one American soldier and three Afghan soldiers died.'}